In [ ]:
%cd /content
!git clone https://github.com/Arpnik/EDU-Chunking-RAG.git
%cd EDU-Chunking-RAG


/content
fatal: destination path 'EDU-Chunking-RAG' already exists and is not an empty directory.
/content/EDU-Chunking-RAG


In [ ]:
%cd /content/EDU-Chunking-RAG

# Make sure pip is recent
!pip install -U pip

# Install this project in editable mode (uses pyproject.toml)
!pip install -e .

# (Optional) If you ever see "ModuleNotFoundError" for something, just pip install it:
# !pip install qdrant-client sentence-transformers transformers tqdm torch


/content/EDU-Chunking-RAG
Obtaining file:///content/EDU-Chunking-RAG
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fever-rag (pyproject.toml) ... done
  Created wheel for fever-rag: filename=fever_rag-0.1.0-0.editable-py3-none-any.whl size=5331 sha256=8dcdb3709e3b7185bbff5996cba7b7d23189e794c79949cbfbbb7b6b2e4219cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-sqnztygd/wheels/5e/26/ba/bb65e0c0db944c0f992ec6ba1c7944f620e2a5ab5b9fd574cb
Successfully built fever-rag
  Attempting uninstall: fever-rag
    Found existing installation: fever-rag 0.1.0
    Uninstalling fever-rag-0.1.0:
      Successfully uninstalled fever-rag-0.1.0


In [ ]:
from google.colab import files

# Upload your `fever_dataset.zip` from your machine
uploaded = files.upload()


Saving reduced_fever_data.zip to reduced_fever_data (1).zip


In [ ]:
%cd /content/EDU-Chunking-RAG

# Just unzip INTO the existing dataset/ folder
!unzip -o reduced_fever_data.zip -d dataset

# Sanity checks
!ls dataset
!ls dataset/reduced_fever_data
!ls dataset/reduced_fever_data/wiki



/content/EDU-Chunking-RAG
Archive:  reduced_fever_data.zip
  inflating: dataset/__MACOSX/._reduced_fever_data  
  inflating: dataset/reduced_fever_data/train.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._train.jsonl  
  inflating: dataset/reduced_fever_data/paper_test.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._paper_test.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._wiki  
  inflating: dataset/reduced_fever_data/paper_dev.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._paper_dev.jsonl  
  inflating: dataset/reduced_fever_data/wiki/filtered_evidence.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/wiki/._filtered_evidence.jsonl  
data_exploration.ipynb	__MACOSX  reduced_fever_data
paper_dev.jsonl  paper_test.jsonl  train.jsonl	wiki
filtered_evidence.jsonl


In [ ]:
from google.colab import files
uploaded = files.upload()  # upload something like edu_segmenter_linear.zip


Saving MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip to MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES (1).zip


In [ ]:
%cd /content/EDU-Chunking-RAG
!mkdir -p edu_segmenter_linear
!unzip -o MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip -d edu_segmenter_linear


/content/EDU-Chunking-RAG
Archive:  MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/vocab.txt  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/adapter_config.json  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/rng_state.pth  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/adapter_model.safetensors  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/tokenizer_config.json  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/scaler.pt  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468

In [ ]:
!ls edu_segmenter_linear


MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES


In [ ]:
%cd /content/EDU-Chunking-RAG
!ls edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES


/content/EDU-Chunking-RAG
best_model	checkpoint-546	checkpoint-702
checkpoint-468	checkpoint-624	checkpoint-780


In [ ]:
MODEL_PATH = "edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/best_model"


In [ ]:
from pathlib import Path

# Project root in Colab
PROJECT_ROOT = Path("/content/EDU-Chunking-RAG")

# Point to the reduced dataset inside the existing dataset/ folder
DATASET_ROOT = PROJECT_ROOT / "dataset" / "reduced_fever_data"
WIKI_DIR_PATH = DATASET_ROOT / "wiki"
CLAIM_FILE_PATH = DATASET_ROOT / "paper_dev.jsonl"

# EDU segmenter model path (adjust folder name if yours is different)
MODEL_PATH_PATH = PROJECT_ROOT / "edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES"  / "best_model"

# String versions for use in !python CLI commands
DATASET_ROOT_STR = str(DATASET_ROOT)
WIKI_DIR = str(WIKI_DIR_PATH)
CLAIM_FILE = str(CLAIM_FILE_PATH)
MODEL_PATH = str(MODEL_PATH_PATH)

print("DATASET_ROOT:", DATASET_ROOT_STR)
print("WIKI_DIR:", WIKI_DIR)
print("CLAIM_FILE:", CLAIM_FILE)
print("MODEL_PATH:", MODEL_PATH)

# Optional sanity check: see which ones actually exist
for p in [DATASET_ROOT, WIKI_DIR_PATH, CLAIM_FILE_PATH, MODEL_PATH_PATH]:
    print(p, "exists:", p.exists())


DATASET_ROOT: /content/EDU-Chunking-RAG/dataset/reduced_fever_data
WIKI_DIR: /content/EDU-Chunking-RAG/dataset/reduced_fever_data/wiki
CLAIM_FILE: /content/EDU-Chunking-RAG/dataset/reduced_fever_data/paper_dev.jsonl
MODEL_PATH: /content/EDU-Chunking-RAG/edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/best_model
/content/EDU-Chunking-RAG/dataset/reduced_fever_data exists: True
/content/EDU-Chunking-RAG/dataset/reduced_fever_data/wiki exists: True
/content/EDU-Chunking-RAG/dataset/reduced_fever_data/paper_dev.jsonl exists: True
/content/EDU-Chunking-RAG/edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/best_model exists: True


In [ ]:
from pathlib import Path

file_path = Path("/content/EDU-Chunking-RAG/com/fever/rag/utils/data_helper.py")
text = file_path.read_text()

# Replace the wrong import with the correct one
text = text.replace(
    "from sympy.printing.pytorch import torch",
    "import torch"
)

file_path.write_text(text)
print("Patched data_helper.py ✅")


Patched data_helper.py ✅


In [ ]:
%cd /content/EDU-Chunking-RAG
!pip install sentence-transformers qdrant-client


/content/EDU-Chunking-RAG


In [ ]:
import os
print(os.path.exists(MODEL_PATH))


True


In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 500 \
  --chunking_overlap 35 \
  --retrieval_strategy top_k \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 03:04:54.852064: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 03:04:54.869472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763867094.890312   14809 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763867094.896632   14809 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763867094.912723   14809 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_token \
  --max_tokens 150 \
  --chunking_overlap 35 \
  --chunk_size 128 \
  --retrieval_strategy top_k \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 02:48:46.579665: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 02:48:46.597223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763866126.618362   10624 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763866126.624737   10624 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763866126.640680   10624 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:

%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 500 \
  --chunking_overlap 75 \
  --retrieval_strategy top_k \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 03:20:02.942599: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 03:20:02.960225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763868002.981196   18643 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763868002.987531   18643 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763868003.003694   18643 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 700 \
  --chunking_overlap 75 \
  --retrieval_strategy top_k \
  --top_k 20

/content/EDU-Chunking-RAG
2025-11-23 03:27:58.601129: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 03:27:58.618545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763868478.639418   20717 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763868478.645701   20717 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763868478.661745   20717 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 700 \
  --chunking_overlap 35 \
  --retrieval_strategy top_k \
  --top_k 20

/content/EDU-Chunking-RAG
2025-11-23 03:34:40.065598: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 03:34:40.083478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763868880.104740   22459 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763868880.111107   22459 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763868880.127354   22459 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_token \
  --max_tokens 150 \
  --chunking_overlap 75 \
  --chunk_size 128 \
  --retrieval_strategy top_k \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 03:41:43.652920: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 03:41:43.670483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763869303.691492   24311 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763869303.697849   24311 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763869303.714104   24311 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_token \
  --max_tokens 150 \
  --chunking_overlap 75 \
  --chunk_size 256 \
  --retrieval_strategy top_k \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 03:58:16.528766: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 03:58:16.546504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763870296.567955   28445 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763870296.574391   28445 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763870296.590809   28445 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_token \
  --max_tokens 150 \
  --chunking_overlap 35 \
  --chunk_size 256 \
  --retrieval_strategy top_k \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 04:13:40.636396: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 04:13:40.654230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763871220.675567   32299 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763871220.682010   32299 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763871220.698427   32299 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type sentence \
  --retrieval_strategy top_k \
  --max_tokens 150 \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 04:22:51.806347: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 04:22:51.825019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763871771.846564   34673 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763871771.853028   34673 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763871771.869469   34673 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type custom_edu \
  --max_tokens 150 \
  --chunking_overlap 2 \
  --retrieval_strategy top_k \
  --model_path "$MODEL_PATH" \
  --top_k 20


/content/EDU-Chunking-RAG
2025-11-23 04:40:43.213093: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 04:40:43.230581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763872843.251340   39265 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763872843.257630   39265 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763872843.273764   39265 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 500 \
  --chunking_overlap 35 \
  --retrieval_strategy threshold \
  --threshold 0.8


/content/EDU-Chunking-RAG
2025-11-23 04:51:24.094348: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 04:51:24.112188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763873484.133636   42031 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763873484.140129   42031 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763873484.156465   42031 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 500 \
  --chunking_overlap 75 \
  --retrieval_strategy threshold \
  --threshold 0.8


/content/EDU-Chunking-RAG
2025-11-23 05:00:14.505481: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 05:00:14.523230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763874014.543974   44333 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763874014.550278   44333 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763874014.566377   44333 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type fixed_char \
  --max_tokens 150 \
  --chunk_size 700 \
  --chunking_overlap 75 \
  --retrieval_strategy threshold \
  --threshold 0.8


/content/EDU-Chunking-RAG
2025-11-23 05:08:34.967235: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 05:08:34.984863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763874515.006072   46508 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763874515.012441   46508 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763874515.028758   46508 computation_placer.cc:177] computation placer already registered. Please check 

In [ ]:
%cd /content/EDU-Chunking-RAG

!python -m com.fever.rag.retriever.retriever_evaluator \
  --qdrant_in_memory True \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --output_file retrieval_evaluation_results.jsonl \
  --claim_file_path dataset/reduced_fever_data/paper_dev.jsonl \
  --chunker_type sentence \
  --retrieval_strategy threshold \
  --max_tokens 150 \
  --threshold 0.8


/content/EDU-Chunking-RAG
2025-11-23 05:16:02.534986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 05:16:02.553360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763874962.574479   48460 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763874962.580927   48460 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763874962.597336   48460 computation_placer.cc:177] computation placer already registered. Please check 